In [22]:
!pip install rasterio -q
!pip install pandas_path --quiet
!pip install line_profiler

import shutil
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas_path import path
from pathlib import Path
from PIL import Image
import rasterio

import os
import random
import numpy.ma as ma
plt.style.use("ggplot")
%matplotlib inline

from IPython.display import HTML, display

!pip install line_profiler

import cv2
import albumentations
from tqdm import tqdm_notebook, tnrange
from itertools import chain
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from sklearn.model_selection import train_test_split

import tensorflow as tf
import keras

from tensorflow.keras.metrics import MeanIoU
import keras.backend as K
from keras.models import Model, load_model
from keras.layers import Input, BatchNormalization, Activation, Dense, Dropout
from keras.layers.core import Lambda, RepeatVector, Reshape
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D, GlobalMaxPool2D
from keras.layers.merge import concatenate, add
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

pd.set_option('max_colwidth', 400)

In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
tf.test.gpu_device_name()

'/device:GPU:0'

## Load data path

In [25]:
img_size = 512

DATA_DIR = Path.cwd().parent / "content/drive/MyDrive/DATA_SCIENCE/data_google_colab/cloud_cover/raw/train"
TRAIN_FEATURES = DATA_DIR / "train_features"
TRAIN_LABELS = DATA_DIR / "train_labels"

BANDS = ["B02", "B03", "B04", "B08"]

train_meta = pd.read_csv(DATA_DIR / "train_metadata_updated.csv", index_col=0)

In [26]:
def add_paths(df, feature_dir, label_dir=None, bands=BANDS):
    """
    Given dataframe with a column for chip_id, returns a dataframe with a column
    added indicating the path to each band's TIF image as "{band}_path", eg "B02_path".
    A column is also added to the dataframe with paths to the label TIF, if the
    path to the labels directory is provided.
    """
    for band in bands:
        df[f"{band}_path"] = ""

    for i in range(len(df)):
      for band in bands:
          df[f"{band}_path"][i] = feature_dir / df["chip_id"][i] / f"{band}.tif"
          #assert df[f"{band}_path"].path.exists().all()
      if label_dir is not None:
          df["label_path"][i] = label_dir / (df["chip_id"][i] + ".tif")
          #assert df["label_path"].path.exists().all()

    return df

In [27]:
train_meta = add_paths(train_meta, TRAIN_FEATURES, TRAIN_LABELS, bands=BANDS)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [28]:
train_meta.head(5)

,chip_id,location,datetime,cloudpath,B02_path,B03_path,B04_path,B08_path,label_path,cloud_cover
0,adwp,Chifunfu,2020-04-29T08:20:47Z,az://./train_features/adwp,/content/drive/MyDrive/DATA_SCIENCE/data_google_colab/cloud_cover/raw/train/train_features/adwp/B02.tif,/content/drive/MyDrive/DATA_SCIENCE/data_google_colab/cloud_cover/raw/train/train_features/adwp/B03.tif,/content/drive/MyDrive/DATA_SCIENCE/data_google_colab/cloud_cover/raw/train/train_features/adwp/B04.tif,/content/drive/MyDrive/DATA_SCIENCE/data_google_colab/cloud_cover/raw/train/train_features/adwp/B08.tif,/content/drive/MyDrive/DATA_SCIENCE/data_google_colab/cloud_cover/raw/train/train_labels/adwp.tif,0.479294
1,adwu,Chifunfu,2020-04-29T08:20:47Z,az://./train_features/adwu,/content/drive/MyDrive/DATA_SCIENCE/data_google_colab/cloud_cover/raw/train/train_features/adwu/B02.tif,/content/drive/MyDrive/DATA_SCIENCE/data_google_colab/cloud_cover/raw/train/train_features/adwu/B03.tif,/content/drive/MyDrive/DATA_SCIENCE/data_google_colab/cloud_cover/raw/train/train_features/adwu/B04.tif,/content/drive/MyDrive/DATA_SCIENCE/data_google_colab/cloud_cover/raw/train/train_features/adwu/B08.tif,/content/drive/MyDrive/DATA_SCIENCE/data_google_colab/cloud_cover/raw/train/train_labels/adwu.tif,0.636658
2,adwz,Chifunfu,2020-04-29T08:20:47Z,az://./train_features/adwz,/content/drive/MyDrive/DATA_SCIENCE/data_google_colab/cloud_cover/raw/train/train_features/adwz/B02.tif,/content/drive/MyDrive/DATA_SCIENCE/data_google_colab/cloud_cover/raw/train/train_features/adwz/B03.tif,/content/drive/MyDrive/DATA_SCIENCE/data_google_colab/cloud_cover/raw/train/train_features/adwz/B04.tif,/content/drive/MyDrive/DATA_SCIENCE/data_google_colab/cloud_cover/raw/train/train_features/adwz/B08.tif,/content/drive/MyDrive/DATA_SCIENCE/data_google_colab/cloud_cover/raw/train/train_labels/adwz.tif,1.000000
3,adxp,Chifunfu,2020-04-29T08:20:47Z,az://./train_features/adxp,/content/drive/MyDrive/DATA_SCIENCE/data_google_colab/cloud_cover/raw/train/train_features/adxp/B02.tif,/content/drive/MyDrive/DATA_SCIENCE/data_google_colab/cloud_cover/raw/train/train_features/adxp/B03.tif,/content/drive/MyDrive/DATA_SCIENCE/data_google_colab/cloud_cover/raw/train/train_features/adxp/B04.tif,/content/drive/MyDrive/DATA_SCIENCE/data_google_colab/cloud_cover/raw/train/train_features/adxp/B08.tif,/content/drive/MyDrive/DATA_SCIENCE/data_google_colab/cloud_cover/raw/train/train_labels/adxp.tif,1.000000
4,aeaj,Chifunfu,2020-04-29T08:20:47Z,az://./train_features/aeaj,/content/drive/MyDrive/DATA_SCIENCE/data_google_colab/cloud_cover/raw/train/train_features/aeaj/B02.tif,/content/drive/MyDrive/DATA_SCIENCE/data_google_colab/cloud_cover/raw/train/train_features/aeaj/B03.tif,/content/drive/MyDrive/DATA_SCIENCE/data_google_colab/cloud_cover/raw/train/train_features/aeaj/B04.tif,/content/drive/MyDrive/DATA_SCIENCE/data_google_colab/cloud_cover/raw/train/train_features/aeaj/B08.tif,/content/drive/MyDrive/DATA_SCIENCE/data_google_colab/cloud_cover/raw/train/train_labels/aeaj.tif,1.000000


## Split the data

In [29]:
test_filenames = pd.read_csv(DATA_DIR / "test_split.csv")['location'].to_list()
test_filenames

['Adelaide',
 'Alto Hama',
 'Angkor Wat',
 'Australia - Central',
 'Australia - North West',
 'Bambesa',
 'Bechar',
 'Bor',
 'Cabo Verdo',
 'Chifunfu',
 'Chingola',
 'Ecuador',
 'Eritrea',
 'Gabon',
 'Ghana',
 'Harare']

In [30]:
df_train = train_meta.loc[~train_meta['location'].isin(test_filenames)].iloc[:, 4:-1]
df_test = train_meta.loc[train_meta['location'].isin(test_filenames)].iloc[:, 4:-1]

print("Test data ratio", round(100*len(df_test)/len(train_meta), 2), "%")

Test data ratio 20.66 %


In [31]:
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
df_train.columns

Index(['B02_path', 'B03_path', 'B04_path', 'B08_path', 'label_path'], dtype='object')

## Prepare loading the data

In [32]:
def load_label(df, image_index):
    image = np.zeros((512,512,1))
    with rasterio.open(df['label_path'][image_index]) as img:            
        image[:,:,0] = img.read(1)
    return image

def load_features(df, image_index):
    image = np.zeros((512,512,4))
    for i, band in enumerate(['B02_path', 'B03_path', 'B04_path', 'B08_path']):
        with rasterio.open(df[band][image_index]) as img:            
            image[:,:,i] = img.read(1)
    return image

def load_multiple_label(df, image_indexes):
    image = np.zeros((len(image_indexes),512,512,1))
    for np_idx, idx in enumerate(image_indexes):
      with rasterio.open(df['label_path'][idx]) as img:            
          image[np_idx,:,:,0] = img.read(1)
    return image

def load_multiple_features(df, image_indexes):
    image = np.zeros((len(image_indexes), 512,512,4))
    for np_idx, idx in enumerate(image_indexes):
      for i, band in enumerate(['B02_path', 'B03_path', 'B04_path', 'B08_path']):
          with rasterio.open(df[band][idx]) as img:            
              image[np_idx, :,:,i] = img.read(1)
    return image

In [33]:
class My_Custom_Generator(tf.keras.utils.Sequence) :
  
  def __init__(self, df, batch_size):
    self.df = df
    self.batch_size = batch_size
    
    
  def __len__(self):
    return (np.ceil(len(self.df) / float(self.batch_size))).astype(np.int)
  
  
  def __getitem__(self, idx):
    batch_indexes_list = self.df.loc[self.df.index.isin(range(idx * batch_size,(idx+1) * batch_size))].index.to_list()
    X = load_multiple_features(self.df, batch_indexes_list)
    y = load_multiple_label(self.df, batch_indexes_list)

    return X,y

In [34]:
df_train = df_train[0:254]
df_test = df_test[0:64]

In [35]:
batch_size = 8

train_batch_generator = My_Custom_Generator(df_train, batch_size)
val_batch_generator = My_Custom_Generator(df_test, batch_size)

## Prepare model

In [36]:
# https://gist.github.com/wassname/7793e2058c5c9dacb5212c0ac0b18a8a
def DiceLoss_square(y_true, y_pred, smooth=1):
  #create the missing data mask
  mask = tf.math.not_equal(y_true, 255)
  #apply the mask
  y_true = tf.boolean_mask(y_true, mask)
  y_pred = tf.boolean_mask(y_pred, mask)

  y_true_f = K.flatten(y_true)
  y_pred_f = K.flatten(y_pred)
  intersection = K.sum(K.abs(y_true_f * y_pred_f))
  return 1-((2. * intersection + smooth) / (K.sum(K.square(y_true_f),-1) + K.sum(K.square(y_pred_f),-1) + smooth))

def DiceLoss(y_true, y_pred, smooth=1):
  y_true_f = K.flatten(y_true)
  y_pred_f = K.flatten(y_pred)
  intersection = K.sum(y_true * y_pred)
  return 1-((2. * intersection + smooth) / (K.sum(y_true) + K.sum(y_pred) + smooth))

In [37]:
def IOU_coef(y_true, y_pred):
  #create the missing data mask
  mask = tf.math.not_equal(y_true, 255)
  #apply the mask
  y_true = tf.boolean_mask(y_true, mask)
  y_pred = tf.boolean_mask(y_pred, mask)

  #make all values > 0.5 a 1 and all others a 0
  y_pred = tf.cast((y_pred > 0.5), dtype=tf.float32)
  #y_pred = tf.math.multiply(tf.math.greater(y_pred, 0.5),1.0)

  y_true_f = K.flatten(y_true)
  y_pred_f = K.flatten(y_pred)
  intersection = K.sum(y_true_f * y_pred_f)
  return (intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + 1.0)

# https://www.youtube.com/watch?v=BNPW1mYbgS4
def IOULoss(y_true, y_pred):
    return -IOU_coef(y_true, y_pred)

In [38]:
def conv2d_block(input_tensor, n_filters, kernel_size=3, batchnorm=True):
    # first layer
    x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
               padding="same")(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation("relu")(x)
    # second layer
    x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
               padding="same")(x)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation("relu")(x)
    return x

In [39]:
def get_unet(input_img, n_filters=16, dropout=0.5, batchnorm=True):
    # contracting path
    c1 = conv2d_block(input_img, n_filters=n_filters*1, kernel_size=3, batchnorm=batchnorm)
    p1 = MaxPooling2D((2, 2)) (c1)
    p1 = Dropout(dropout*0.5)(p1)

    c2 = conv2d_block(p1, n_filters=n_filters*2, kernel_size=3, batchnorm=batchnorm)
    p2 = MaxPooling2D((2, 2)) (c2)
    p2 = Dropout(dropout)(p2)

    c3 = conv2d_block(p2, n_filters=n_filters*4, kernel_size=3, batchnorm=batchnorm)
    p3 = MaxPooling2D((2, 2)) (c3)
    p3 = Dropout(dropout)(p3)

    c4 = conv2d_block(p3, n_filters=n_filters*8, kernel_size=3, batchnorm=batchnorm)
    p4 = MaxPooling2D(pool_size=(2, 2)) (c4)
    p4 = Dropout(dropout)(p4)
    
    c5 = conv2d_block(p4, n_filters=n_filters*16, kernel_size=3, batchnorm=batchnorm)
    
    # expansive path
    u6 = Conv2DTranspose(n_filters*8, (3, 3), strides=(2, 2), padding='same') (c5)
    u6 = concatenate([u6, c4])
    u6 = Dropout(dropout)(u6)
    c6 = conv2d_block(u6, n_filters=n_filters*8, kernel_size=3, batchnorm=batchnorm)

    u7 = Conv2DTranspose(n_filters*4, (3, 3), strides=(2, 2), padding='same') (c6)
    u7 = concatenate([u7, c3])
    u7 = Dropout(dropout)(u7)
    c7 = conv2d_block(u7, n_filters=n_filters*4, kernel_size=3, batchnorm=batchnorm)

    u8 = Conv2DTranspose(n_filters*2, (3, 3), strides=(2, 2), padding='same') (c7)
    u8 = concatenate([u8, c2])
    u8 = Dropout(dropout)(u8)
    c8 = conv2d_block(u8, n_filters=n_filters*2, kernel_size=3, batchnorm=batchnorm)

    u9 = Conv2DTranspose(n_filters*1, (3, 3), strides=(2, 2), padding='same') (c8)
    u9 = concatenate([u9, c1], axis=3)
    u9 = Dropout(dropout)(u9)
    c9 = conv2d_block(u9, n_filters=n_filters*1, kernel_size=3, batchnorm=batchnorm)
    
    outputs = Conv2D(1, (1, 1), activation='sigmoid') (c9)
    model = Model(inputs=[input_img], outputs=[outputs])
    return model

In [40]:
input_img = Input((img_size, img_size, 4), name='img')
model = get_unet(input_img, n_filters=16, dropout=0.05, batchnorm=True)

model.compile(optimizer=Adam(), loss=DiceLoss_square, metrics=[IOU_coef])
#model.summary()

In [41]:
callbacks = [
    EarlyStopping(patience=10, verbose=1),
    ReduceLROnPlateau(factor=0.1, patience=3, min_lr=0.00001, verbose=1),
    ModelCheckpoint('unet_diceloss_square.h5', verbose=1, save_best_only=True, save_weights_only=False)
]

In [42]:
def fit_model():
  model.fit(train_batch_generator,
            steps_per_epoch = int(len(train_batch_generator) // batch_size),
            epochs = 1,
            verbose = 1,
            callbacks=callbacks,
            validation_data = val_batch_generator,
            validation_steps = int(len(val_batch_generator) // batch_size))

In [46]:
%load_ext line_profiler
%lprun -f fit_model fit_model()

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler
4/4 [==============================] - ETA: 0s - loss: 0.1171 - IOU_coef: 0.7978 
Epoch 00001: val_loss did not improve from 0.06624
4/4 [==============================] - 85s 27s/step - loss: 0.1171 - IOU_coef: 0.7978 - val_loss: 0.0717 - val_IOU_coef: 0.8634 - lr: 0.0010


In [44]:
model.fit(train_batch_generator,
            steps_per_epoch = int(len(train_batch_generator) // batch_size),
            epochs = 1,
            verbose = 1,
            callbacks=callbacks,
            validation_data = val_batch_generator,
            validation_steps = int(len(val_batch_generator) // batch_size))

4/4 [==============================] - ETA: 0s - loss: 0.1904 - IOU_coef: 0.6764 
Epoch 00001: val_loss improved from inf to 0.06771, saving model to unet_diceloss_square.h5
4/4 [==============================] - 156s 42s/step - loss: 0.1904 - IOU_coef: 0.6764 - val_loss: 0.0677 - val_IOU_coef: 0.8629 - lr: 0.0010
